In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

In [3]:
conn = create_connection('sqlite_db_pythonsqlite.db')

2.6.0


In [4]:
cur = conn.cursor()

In [5]:
# Q10: Produce a list of facilities with a total revenue less than 1000. 
# The output of facility name and total revenue, sorted by revenue. 
# Rememberthat there's a different cost for guests and members!
Q10 = """
SELECT f.Name as FacilityName,
    SUM(CASE WHEN memid = 0 THEN guestcost*slots
        ELSE membercost*slots
        END) AS total_revenue
FROM 
    Facilities AS f
INNER JOIN 
    Bookings AS b
ON 
    f.facid = b.facid
GROUP BY 
    f.Name
HAVING 
    total_revenue < 1000
ORDER BY 
    total_revenue
"""
cur.execute(Q10)

In [6]:
rows = cur.fetchall()
pd.DataFrame(rows, columns = ['facility','total_revenue'])

,facility,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [7]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
Q11 = """
SELECT 
    m1.memid, 
    (m1.surname || ' ' || m1.firstname) AS Name, 
    (m2.surname || ' ' || m2.firstname) AS RecommendedBy
FROM 
    Members AS m1,
    Members AS m2
WHERE 
    m1.recommendedby = m2.memid
ORDER BY
    Name;
"""
cur.execute(Q11)

In [8]:
rows1 = cur.fetchall()
pd.DataFrame(rows1, columns = ['memid','Name','RecommendedBy'])

,memid,Name,RecommendedBy
0,15,Bader Florence,Stibbons Ponder
1,12,Baker Anne,Stibbons Ponder
2,16,Baker Timothy,Farrell Jemima
3,8,Boothe Tim,Rownam Tim
4,5,Butters Gerald,Smith Darren
5,22,Coplin Joan,Baker Timothy
6,36,Crumpet Erica,Smith Tracy
7,7,Dare Nancy,Joplette Janice
8,20,Genting Matthew,Butters Gerald
9,35,Hunt John,Purview Millicent


In [9]:
# Q12: Find the facilities with their usage by member, but not guests
Q12 ="""
SELECT DISTINCT
    f.name AS Facility,
    (m.surname || ' ' || m.firstname) AS Members
FROM 
    Bookings AS b,
    Members AS m,
    Facilities AS f
WHERE 
    m.memid != 0 AND
    b.memid = m.memid AND
    b.facid = f.facid
ORDER BY
    f.name;
"""
cur.execute(Q12)

In [10]:
rows2 = cur.fetchall()
pd.DataFrame(rows2, columns = ['Facility','Members'])

,Facility,Members
0,Badminton Court,Smith Darren
1,Badminton Court,Butters Gerald
2,Badminton Court,Smith Tracy
3,Badminton Court,Dare Nancy
4,Badminton Court,Rownam Tim
...,...,...
197,Tennis Court 2,Smith Jack
198,Tennis Court 2,Purview Millicent
199,Tennis Court 2,Rumney Henrietta
200,Tennis Court 2,Hunt John


In [11]:
# Q13: Find the facilities usage by month, but not guests
Q13 = """
SELECT DISTINCT 
    strftime('%m',starttime) AS Month,
    f.name AS Facility
FROM 
    Bookings AS b,
    Members AS m,
    Facilities AS f
WHERE 
    m.memid != 0 AND
    b.memid = m.memid AND
    b.facid = f.facid
ORDER BY
    Month;
"""
cur.execute(Q13)

In [12]:
rows3 = cur.fetchall()
pd.DataFrame(rows3, columns = ['Month','Facility'])

,Month,Facility
0,07,Table Tennis
1,07,Massage Room 1
2,07,Snooker Table
3,07,Pool Table
4,07,Tennis Court 1
5,07,Squash Court
6,07,Badminton Court
7,07,Tennis Court 2
8,07,Massage Room 2
9,08,Tennis Court 1
